# Dynamic Efficient Case

In [53]:
import Pkg; Pkg.activate(joinpath(@__DIR__, ".."))
Pkg.instantiate()
using Revise
using Aiyagari
using StructArrays
using ProgressMeter
using DelimitedFiles
using Roots
using Plots
using LaTeXStrings

In [54]:
pgfplotsx();
Threads.nthreads()

4

In [55]:
ProgressMeter.ijulia_behavior(:clear);
default(label = "", lw = 2, dpi = 300, left_margin = 0Plots.mm, format=:svg);

In [56]:
_LOAD_GUESSES = true # load the initial starting guesses for zeros from disk
_SAVE_GUESSES = false # save zeros to disk
_ITERS = 1

# Setting _LOAD_GUESSES to false will recompute all the calculations from scratch. At that point, 
# set _ITERS to a higher number. 

1

## The benchmark calibration 

In [57]:
#' The parameters for technology and households.
e = let 
    #Household:
    ar1 = 0.9695
    sigmaP = sqrt(0.0384)/(1.2)
    sigmaIID = sqrt(0.0522)/(1.2)
    P, z_vals = calibration(5, 2 , ar1, sigmaP, sigmaIID)

    ies = 1.0
    crra = 5.5
    β = 0.993
    
    hh = Household(u = EZ(ies = ies, ra = crra), grid_points = 5_000,
        v = GHH(θ = 1.0, ν = 0.2), P = P, z_grid = z_vals, β = β, a_max = 10.0)

    #Technology
    δ = 0.1
    μ = 1.4
    α = .3
    #ρ = 0.7
    A_μ = 0.2
    # t_μ = MarkupTechnology(f = CES(α = α, ρ = 0.7), δ = δ, μ = μ, A = A_μ)
    t_μ = MarkupTechnology(f = CobbDouglas(α = α), δ = δ, μ = μ, A = A_μ)

    Economy(h = hh, t = t_μ)
end

Economy: EZ(ies = 1.0, ra = 5.5), β=0.993, v=GHH(θ = 1.0, ν = 0.2), z_grid=[0.17222630362336605..3.621762275547772], P=[..], pts=5000, a_max=10.0, a_min=0.0, CobbDouglas(α=0.3), A=0.2, δ=0.1, μ=1.4, m=0.0, X=0.0

In [58]:
# Solve laissez-faire economy
@time laissez_faire = let r_range = (-0.0170, -0.0169)
    solve_laissez_faire(e; 
        r_range = r_range, 
        tol =  (value_function = 1e-10, distribution = 1e-13)
    ) 
end 

Progress:  16 	 Time: 0:03:53
  r:       -0.016951710071023102
  excess:  1.1102230246251565e-16

277.242757 seconds (794.17 M allocations: 12.814 GiB, 3.65% gc time, 17.86% compilation time)


r=-0.016951710071023102, b=0.0, k=0.5596544237494959, transfer=0.0,  y=0.21689893327005233, Economy: EZ(ies = 1.0, ra = 5.5), β=0.993, v=GHH(θ = 1.0, ν = 0.2), z_grid=[0.17222630362336605..3.621762275547772], P=[..], pts=5000, a_max=10.0, a_min=0.0, CobbDouglas(α=0.3), A=0.2, δ=0.1, μ=1.4, m=0.0, X=0.0

## Transitions

In [59]:
T = 100  # period of adjustment
H = 50   # stationary part 

50

## Transition with constant k and smooth b

We target a debt of 59% percent of output

In [60]:
b_target = laissez_faire.y * 0.60
k_target = laissez_faire.k

0.5596544237494959

In [ ]:
@time final_eq_1 = solve_new_stationary_equilibrium_given_k_b(
    laissez_faire,
    k_target, 
    b_target; 
    r_range = (laissez_faire.r, -0.010),
    tol = (value_function = 1e-10, distribution = 1e-13)
)

In [ ]:
# Smooth debt policy
ρB = 0.9
b_list = Array{Float64,1}(undef, T + H)
b_list[1] = 0.0
b_list[2] = laissez_faire.y * 0.05
b_list[T:end] .= b_target
for i in 3:T-1
    b_list[i] = b_list[2] * ρB^(i-2) + (1 - ρB^(i-2)) * b_target   
end

In [ ]:
# k remains constant
k_list = [laissez_faire.k for _ in b_list]; 

Computes the transition 

In [ ]:
r_path = nothing 
if _LOAD_GUESSES
    r_path = try 
        # load the transfer vector from previous iterations 
        readdlm(joinpath(@__DIR__,"..", "output", "tmp_calcs", "tmp_r_001.txt"))
    catch 
        nothing 
    end
end;

In [ ]:
@time transition = solve_transition(
    laissez_faire, 
    final_eq_1, 
    k_list, 
    b_list; 
    init_r_path = r_path, 
    iterations = _ITERS);

In [ ]:
_SAVE_GUESSES && open(joinpath(@__DIR__, "..", "output", "tmp_calcs", "tmp_r_001.txt"), "w") do io
    writedlm(io, transition.path.r)
end

### Plots

In [ ]:
plot(transition.path.transfer, fill = 0, alpha = 0.3,)
plot!(transition.path.transfer, color=1)
hline!([final_eq_1.transfer], lw = 1, style = :dash)

In [ ]:
plot(transition.path.r, legend=false)
hline!([laissez_faire.r], lw = 1)
hline!([final_eq_1.r], lw = 1)

## Transition with increasing k and increasing b

We go to the golden rule k -- slowly.

In [ ]:
k_golden = get_golden_k(e.t, laissez_faire.n)
ρK = 0.95
k_list_2 = similar(k_list)
for i in eachindex(k_list_2)
    k_list_2[i] = laissez_faire.k * ρK^(i-1) + (1 - ρK^(i-1)) * k_golden
end 

The debt path and its target remains as in the previous case

In [ ]:
b_list_2 = b_list
b_target_2 = b_list[end]

Computing the new stationary equilibrium in this case:

In [ ]:
@time final_eq_2 = solve_new_stationary_equilibrium_given_k_b(
    laissez_faire,
    k_list_2[end], 
    b_target_2; 
    r_range = (laissez_faire.r, -0.010),
    tol = (value_function = 1e-10, distribution = 1e-13)
)

Computing the transition:

In [ ]:
r_path_2 = nothing 
if _LOAD_GUESSES
    r_path_2 = try 
        # load the transfer vector from previous iterations 
        readdlm(joinpath(@__DIR__, "..", "output", "tmp_calcs", "tmp_r_002.txt"))
    catch b
        nothing 
    end
end;

In [ ]:
@time transition_2 = solve_transition(
    laissez_faire, 
    final_eq_2, 
    k_list_2, 
    b_list_2; 
    init_r_path = r_path_2,
    beta = -0.02,
    m = 10,
    iterations = _ITERS
);

In [ ]:
_SAVE_GUESSES && open(joinpath(@__DIR__, "..", "output", "tmp_calcs", "tmp_r_002.txt"), "w") do io
    writedlm(io, transition_2.path.r)
end

### Plots

In [ ]:
plot(transition_2.path.transfer, fill = 0, alpha = 0.3)
plot!(transition_2.path.transfer, color = 1)
hline!([0, final_eq_2.transfer], lw = 1)

In [ ]:
plot(transition_2.path.r, legend=false)
hline!([laissez_faire.r], lw = 1)
hline!([final_eq_2.r])

#### Comparison with previous case

In [ ]:
f1 = plot(transition_2.path.v[1] ./ laissez_faire.v .- transition.path.v[1] ./ laissez_faire.v, 
    legend = :bottom,
    title = "Welfare gain with k increasing wrt to k fixed")
f2 = plot(transition.path.v[1] ./ laissez_faire.v .- 1)
plot(f1, f2, size = (1000, 400))

In [ ]:
plot(transition_2.path.transfer, label="new")
plot!(transition.path.transfer, label="old")

In [ ]:
plot(transition_2.path.r, label="new")
plot!(transition.path.r, label="old")

In [ ]:
plot(
    [ -(1 + r) * b + bprime for (b, bprime, r) in
        zip(transition.path.b, transition.path.b[2:end], transition.path.r)],
    label = "old"
)
plot!(
    [ -(1 + r) * b + bprime for (b, bprime, r) in
        zip(transition_2.path.b, transition_2.path.b[2:end], transition_2.path.r)],
    label = "new"
)

## Transition with increasing k and no debt

No debt: 

In [ ]:
b_list_3 = [0.0 for _ in k_list]
b_target_3 = b_list_3[end];

But the same increase in k as before:

In [ ]:
k_list_3 = k_list_2;

Computing the new stationary equilibrium

In [ ]:
@time final_eq_3 = solve_new_stationary_equilibrium_given_k_b(
    laissez_faire,
    k_list_3[end], 
    b_target_3; 
    r_range = (laissez_faire.r, -0.010),
    tol = (value_function = 1e-10, distribution = 1e-13)
)

In [ ]:
r_path_3 = nothing 
if _LOAD_GUESSES 
    r_path_3 = try 
        readdlm(joinpath(@__DIR__, "..", "output", "tmp_calcs", "tmp_r_003.txt"))
    catch 
        nothing 
    end
end;

In [ ]:
@time transition_3 = solve_transition(
    laissez_faire, 
    final_eq_3, 
    k_list_3, 
    b_list_3; 
    init_r_path = r_path_3,
    beta = -0.02,
    m = 10,
    iterations = _ITERS
);

In [ ]:
_SAVE_GUESSES && open(joinpath(@__DIR__, "..", "output", "tmp_calcs", "tmp_r_003.txt"), "w") do io
    writedlm(io, transition_3.path.r)
end

### Plots 

In [ ]:
plot(transition_3.path.transfer, 
    fill = 0, alpha = 0.3, label = "transfers", legend = :bottom)
plot!(transition_3.path.transfer, color = 1) 
hline!([0, final_eq_3.transfer], lw = 1)

In [ ]:
plot(transition_3.path.r, legend = :bottom, label = "r")
hline!([laissez_faire.r, final_eq_3.r], lw = 1)

## Statistics  

In [ ]:
println("INITIAL STEADY STATE")
println("=====================")
summary_statics(laissez_faire)

In [ ]:
println("\nFINAL STEADY STATE -- CONSTANT K AND DEBT")
println("=====================")
summary_statics(final_eq_1, 
    laissez_faire = laissez_faire, 
    path = transition.path)

In [ ]:
println("FINAL STEADY STATE -- GOLDEN K AND DEBT")
println("=====================")
summary_statics(final_eq_2, 
    laissez_faire = laissez_faire,
    path = transition_2.path)

## Figures for paper

### Transition plots

In [ ]:
f1 = plot(do_plots(transition, laissez_faire)..., size = (1000, 500))

In [ ]:
f2 = plot(do_plots(transition_2, laissez_faire)..., size = (1000, 500))

In [ ]:
f3 = plot(do_plots(transition_3, laissez_faire)..., size = (1000, 500))

### Steady state segniorage/transfer plot

We need to increase a_max so that it does not bind as we increase b.

In [ ]:
# Increasing the amax so that it doesn't bind
e_2 = let 
    #Household:
    ar1 = 0.9695
    sigmaP = sqrt(0.0384)/(1.2)
    sigmaIID = sqrt(0.0522)/(1.2)
    P, z_vals = calibration(5, 2 , ar1, sigmaP, sigmaIID)

    ies = 1.0
    crra = 5.5
    β = 0.993
    
    hh = Household(u = EZ(ies = ies, ra = crra), grid_points = 5_000,
        v = GHH(θ = 1.0, ν = 0.2), P = P, z_grid = z_vals, β = β, a_max = 50.0)

    #Technology
    δ = 0.1
    μ = 1.4
    α = .3
    #ρ = 0.7
    A_μ = 0.2
    # t_μ = MarkupTechnology(f = CES(α = α, ρ = 0.7), δ = δ, μ = μ, A = A_μ)
    t_μ = MarkupTechnology(f = CobbDouglas(α = α), δ = δ, μ = μ, A = A_μ)

    Economy(h = hh, t = t_μ)
end

In [ ]:
# Solve laissez-faire economy
@time laissez_faire_2 = let r_range = (-0.0175, -0.016)
    solve_laissez_faire(e_2; 
        r_range = r_range, 
        tol =  (value_function = 1e-10, distribution = 1e-13)
    ) 
end

In [ ]:
r_guess = nothing 
if _LOAD_GUESSES
    r_guess = try 
        # load the transfer vector from previous iterations 
        readdlm(joinpath(@__DIR__, "..", "output", "tmp_calcs", "tmp_ss_b_r.txt"))
    catch 
        nothing 
    end
end;

In [ ]:
out = let
    range = [laissez_faire_2.r, 0.0]
    f = (b, i) -> begin 
        print(i, " ", b, ", ")
        if !isnothing(r_guess)
            myrange = [r_guess[i, 2] - 1e-5, r_guess[i, 2] + 1e-5]
        else
            myrange = range
        end
        sol = solve_new_stationary_equilibrium_given_k_b(
            laissez_faire_2,
            laissez_faire_2.k, 
            b * laissez_faire_2.y; 
            r_range = myrange,
            tol = (value_function = 1e-7, distribution = 1e-7),
            verbose = false,
            find_zero_args = (xatol = 1e-5, maxevals = _ITERS)
        )
        range[2] = sol.r  # iterating down -- change top value of range 
        sol
    end
    [f(i, b) for (b, i) in enumerate(reverse(0.2:0.2:7.0))]
end
push!(out, laissez_faire_2);

In [ ]:
_SAVE_GUESSES && open(joinpath(@__DIR__, "..", "output", "tmp_calcs", "tmp_ss_b_r.txt"), "w") do io
    writedlm(io, [(eq.b, eq.r) for eq in out])
end

In [ ]:
f4 = let 
    by = [eq.b / eq.y for eq in out]
    rb = [-eq.r * eq.b / eq.y for eq in out]
    deltark = [(eq.r - laissez_faire_2.r) * eq.k / eq.y for eq in out]
    plot(by, rb,  label = L"- r  b / y", legend = :bottom, size = (1000/3, 500/2))
    plot!(by, deltark, fillrange = rb, color = 1, alpha = 0.2)
    plot!(by, deltark, color = 2, label = L" (r - r_0)  k_0 / y")
end 

## Exporting the figures

In [ ]:
savefig(f1, joinpath(@__DIR__, "..", "output", "figures", "transition_efficient_fixed_k.pdf"))

In [ ]:
savefig(f2, joinpath(@__DIR__, "..", "output", "figures", "transition_efficient_golden_k.pdf"))

In [ ]:
savefig(f3, joinpath(@__DIR__, "..", "output", "figures", "transition_efficient_no_debt.pdf"))

In [ ]:
savefig(f4, joinpath(@__DIR__, "..", "output", "figures", "steady_state_transfers.pdf"))